. Custom Trained CNN
a. Train a CNN model using Keras Tensorflow ground up
b. Tune the models hyperparameters
i. #Layers
ii. #Units
iii. Epochs
iv. Learning Rate
v. Optimizers etc.
c. Track the following metrics on Train and Validation sets
i. Loss
ii. Accuracy
iii. AUC
iv. Time taken to tune


In [1]:
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train , y_train) , (x_test , y_test) = load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
x_train.dtype

dtype('uint8')

In [5]:
x_train = x_train.astype(float)/255
x_test = x_test.astype(float)/255

In [6]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [7]:
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

# Model building

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D ,Flatten,Dense,MaxPool2D,Activation,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [26]:
model.compile(optimizer = Adam(learning_rate = 0.001),loss = 'categorical_crossentropy' , metrics = ['accuracy',AUC()])

In [27]:
model.fit(x = x_train , y=y_train_ohe , epochs = 10 , validation_data = (x_test , y_test_ohe))

Epoch 1/10
1563/1563 [==============================] - 32s 20ms/step - loss: 1.4422 - accuracy: 0.4875 - auc: 0.8796 - val_loss: 1.2610 - val_accuracy: 0.5532 - val_auc: 0.9090
Epoch 2/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.1571 - accuracy: 0.5958 - auc: 0.9237 - val_loss: 1.1174 - val_accuracy: 0.6036 - val_auc: 0.9287
Epoch 3/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.0349 - accuracy: 0.6388 - auc: 0.9390 - val_loss: 1.1358 - val_accuracy: 0.6059 - val_auc: 0.9270
Epoch 4/10
1563/1563 [==============================] - 29s 19ms/step - loss: 0.9401 - accuracy: 0.6721 - auc: 0.9495 - val_loss: 1.0753 - val_accuracy: 0.6280 - val_auc: 0.9340
Epoch 5/10
1563/1563 [==============================] - 29s 19ms/step - loss: 0.8641 - accuracy: 0.6981 - auc: 0.9571 - val_loss: 1.0277 - val_accuracy: 0.6488 - val_auc: 0.9396
Epoch 6/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.8011 - accuracy: 0.7205 - auc:

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 7200)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               921728    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 923914 (3.52 MB)
Trainable params: 923914 (3.52 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [29]:
history = model.history.history

In [31]:
train_loss = history['loss'][-1]
train_accuracy = history['accuracy'][-1]
train_auc = history['auc'][-1]
val_loss = history['val_loss'][-1]
val_accuracy = history['val_accuracy'][-1]
val_auc = history['val_auc'][-1]

In [35]:
print("Custom CNN Model:")
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)
print("Training AUC:", train_auc)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
print("Validation AUC:", val_auc)

Custom CNN Model:
Training Loss: 0.5936180353164673
Training Accuracy: 0.7914400100708008
Training AUC: 0.9791557192802429
Validation Loss: 1.1018283367156982
Validation Accuracy: 0.6510999798774719
Validation AUC: 0.9348707795143127


#  2. Using a Pretrained Model
a. Load a pretrained model like VGG16 or Resnet50
b. Finetune the last 2/3 layers using a small part of the CIFAR10 dataset (20 percentage or
so)
c. Track the following metrics on Train and Validation sets
i. Loss
ii. Accuracy
iii. AUC
iv. Time taken to tune
3. Compare the results
a. Now compare the results and populate the following table
Approach Accuracy AUC Time Taken
Custom CNN Model
Pretrained Model
Document your understanding from the above results

In [38]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

In [39]:
# Load a pretrained model (VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [40]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(10, activation='softmax')(x)

In [44]:
# Create the full model
import tensorflow as tf
pretrained_model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [45]:
# Freeze the layers except the last 2/3 layers
for layer in base_model.layers[:-6]:
    layer.trainable = False

In [46]:
# Compile the model
pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', AUC()])

In [47]:
# Prepare the data for pretrained model
x_train_pretrained, _, y_train_pretrained, _ = train_test_split(x_train, y_train, test_size=0.8, random_state=42)

In [48]:
import time

In [50]:
# Fine-tune the pretrained model
pretrained_history = pretrained_model.fit(x_train_pretrained, y_train_pretrained, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10


ValueError: ignored

In [ ]:
# Track metrics
pretrained_train_loss = pretrained_history.history['loss'][-1]
pretrained_train_accuracy = pretrained_history.history['accuracy'][-1]
pretrained_train_auc = pretrained_history.history['auc'][-1]
pretrained_val_loss = pretrained_history.history['val_loss'][-1]
pretrained_val_accuracy = pretrained_history.history['val_accuracy'][-1]
pretrained_val_auc = pretrained_history.history['val_auc'][-1]

In [ ]:
# Print metrics and time taken
print("\nPretrained Model:")
print("Training Loss:", pretrained_train_loss)
print("Training Accuracy:", pretrained_train_accuracy)
print("Training AUC:", pretrained_train_auc)
print("Validation Loss:", pretrained_val_loss)
print("Validation Accuracy:", pretrained_val_accuracy)
print("Validation AUC:", pretrained_val_auc)
print("Time taken:", pretrained_training_time)